In [4]:
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [45]:
path_df_fin = "pickles/raw_dataset_fin.pickle"
path_df_inv = "pickles/raw_dataset_inv.pickle"

with open(path_df_fin, 'rb') as data:
    df_fin = pickle.load(data)
    
with open(path_df_inv, 'rb') as data:
    df_inv = pickle.load(data)

In [46]:
# X_train
with open('pickles/X_train_inv.pickle', 'rb') as data:
    X_train_inv = pickle.load(data)
    
# X_test    
with open('pickles/X_test_inv.pickle', 'rb') as data:
    X_test_inv = pickle.load(data)
    
# y_train
with open('pickles/y_train_inv.pickle', 'rb') as data:
    y_train_inv = pickle.load(data)
    
# y_test
with open('pickles/y_test_inv.pickle', 'rb') as data:
    y_test_inv = pickle.load(data)
    
# features_train
with open('pickles/features_train_inv.pickle', 'rb') as data:
    features_train_inv = pickle.load(data)

# labels_train
with open('pickles/labels_train_inv.pickle', 'rb') as data:
    labels_train_inv = pickle.load(data)

# features_test
with open('pickles/features_test_inv.pickle', 'rb') as data:
    features_test_inv = pickle.load(data)

# labels_test
with open('pickles/labels_test_inv.pickle', 'rb') as data:
    labels_test_inv = pickle.load(data)

In [47]:
# X_train
with open('pickles/X_train_fin.pickle', 'rb') as data:
    X_train_fin = pickle.load(data)
    
# X_test    
with open('pickles/X_test_fin.pickle', 'rb') as data:
    X_test_fin = pickle.load(data)
    
# y_train
with open('pickles/y_train_fin.pickle', 'rb') as data:
    y_train_fin = pickle.load(data)
    
# y_test
with open('pickles/y_test_fin.pickle', 'rb') as data:
    y_test_fin = pickle.load(data)
    
# features_train
with open('pickles/features_train_fin.pickle', 'rb') as data:
    features_train_fin = pickle.load(data)

# labels_train
with open('pickles/labels_train_fin.pickle', 'rb') as data:
    labels_train_fin = pickle.load(data)

# features_test
with open('pickles/features_test_fin.pickle', 'rb') as data:
    features_test_fin = pickle.load(data)

# labels_test
with open('pickles/labels_test_fin.pickle', 'rb') as data:
    labels_test_fin = pickle.load(data)

In [48]:
# TF-IDF object
with open('pickles/tfidf.pickle', 'rb') as data:
    tfidf = pickle.load(data)

In [49]:
print(features_train_fin.shape)
print(features_test_fin.shape)
print('\n')
print(features_train_inv.shape)
print(features_test_inv.shape)

(223, 1404)
(40, 1404)


(238, 1469)
(42, 1469)


# FINTECH

In [50]:
# Create the parameter grid based on the results of random search 
bootstrap = [False]
max_depth = [3, 4, 5, 6, 7]
max_features = ['sqrt']
min_samples_leaf = [1, 2]
min_samples_split = [2, 3, 5]
n_estimators = [800]

param_grid = {
    'bootstrap': bootstrap,
    'max_depth': max_depth,
    'max_features': max_features,
    'min_samples_leaf': min_samples_leaf,
    'min_samples_split': min_samples_split,
    'n_estimators': n_estimators
}

# Create a base model
rfc = RandomForestClassifier(random_state=8)

# Manually create the splits in CV in order to be able to fix a random_state (GridSearchCV doesn't have that argument)
cv_sets = ShuffleSplit(n_splits = 3, test_size = .33, random_state = 8)

# Instantiate the grid search model
grid_search_fin = GridSearchCV(estimator=rfc, 
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=cv_sets,
                           verbose=1)

# Fit the grid search to the data
grid_search_fin.fit(features_train_fin, labels_train_fin)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:  1.4min finished


GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=8, test_size=0.33, train_size=None),
             error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_...
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=8,
                                              verbose=0, warm_start=False),
  

In [51]:
print("The best hyperparameters from Grid Search are:")
print(grid_search_fin.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(grid_search_fin.best_score_)

The best hyperparameters from Grid Search are:
{'bootstrap': False, 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 800}

The mean accuracy of a model with these hyperparameters is:
0.7477477477477478


In [52]:
best_rfc_fin = grid_search_fin.best_estimator_

In [53]:
best_rfc_fin

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=5, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=800,
                       n_jobs=None, oob_score=False, random_state=8, verbose=0,
                       warm_start=False)

In [54]:
best_rfc_fin.fit(features_train_fin, labels_train_fin)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=5, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=800,
                       n_jobs=None, oob_score=False, random_state=8, verbose=0,
                       warm_start=False)

In [55]:
rfc_pred_fin = best_rfc_fin.predict(features_test_fin)

In [56]:
# Training accuracy
print("The training accuracy is: ")
print(accuracy_score(labels_train_fin, best_rfc_fin.predict(features_train_fin)))

The training accuracy is: 
0.8565022421524664


In [57]:
# Test accuracy
print("The test accuracy is: ")
print(accuracy_score(labels_test_fin, rfc_pred_fin))

The test accuracy is: 
0.8


# INVESTMENT

In [30]:
# Create the parameter grid based on the results of random search 
bootstrap = [False]
max_depth = [30, 40, 50]
max_features = ['sqrt']
min_samples_leaf = [1, 2, 4]
min_samples_split = [5, 10, 15]
n_estimators = [800]

param_grid = {
    'bootstrap': bootstrap,
    'max_depth': max_depth,
    'max_features': max_features,
    'min_samples_leaf': min_samples_leaf,
    'min_samples_split': min_samples_split,
    'n_estimators': n_estimators
}

# Create a base model
rfc = RandomForestClassifier(random_state=8)

# Manually create the splits in CV in order to be able to fix a random_state (GridSearchCV doesn't have that argument)
cv_sets = ShuffleSplit(n_splits = 3, test_size = .33, random_state = 8)

# Instantiate the grid search model
grid_search_inv = GridSearchCV(estimator=rfc, 
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=cv_sets,
                           verbose=1)

# Fit the grid search to the data
grid_search_inv.fit(features_train_inv, labels_train_inv)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 